In [1]:
import pandas as pd
import numpy as np
import pymc as pm
from bokeh.io import output_notebook
from bokeh.models import BoxAnnotation, Label, Legend, Span
from bokeh.palettes import brewer
from bokeh.plotting import figure, show

output_notebook()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

In [3]:
df = pd.read_csv('nino12.long.anom.csv')
df.columns = ['Date', 'NINA12']
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'nino12.long.anom.csv'

In [45]:
df['NINA12'] = df['NINA12'].replace(-9999, np.nan)
df['NINA12'].isna().sum()

np.int64(4)

In [47]:
df = df.dropna()
df['NINA12'].isna().sum()

np.int64(0)

In [48]:
df.index = pd.to_datetime(df.Date)
df.head()

,Date,NINA12
Date,,
1870-01-01,1870-01-01,-1.50
1870-02-01,1870-02-01,-0.96
1870-03-01,1870-03-01,-0.65
1870-04-01,1870-04-01,-0.32
1870-05-01,1870-05-01,-0.64


In [65]:
df.tail()

,Date,NINA12,t,y_n
Date,,,,
2025-04-01,2025-04-01,0.04,155.350685,1.697686
2025-05-01,2025-05-01,0.21,155.432877,1.885093
2025-06-01,2025-06-01,0.44,155.517808,2.138644
2025-07-01,2025-07-01,0.29,155.600000,1.973285
2025-08-01,2025-08-01,0.04,155.684932,1.697686


In [50]:
def dates_to_idx(timelist):
    reference_time = pd.to_datetime("1958-03-15")
    t = (timelist - reference_time) / pd.Timedelta(365, "D")
    return np.asarray(t)

t = dates_to_idx(df.index)

# Normalise the output values
y = df['NINA12'].values
first_nina= y[0]
std_nina = np.std(y)
y_n = (y - first_nina) / std_nina

df = df.assign(t=t)
data_monthly = df.assign(y_n=y_n)

In [51]:
p = figure(
    x_axis_type='datetime',
    title='NINO 1.2 Index over time',
    width=550,
    height=350
)
p.yaxis.axis_label = 'NINO1.2'
p.xaxis.axis_label = 'Date'

zeroline = Span(location=0, dimension='width', line_color='red', line_dash='dashed', line_width=2)
p.add_layout(zeroline)

p.line(df.index, df['NINA12'], line_width=2, line_color='black', alpha=0.5)

show(p)

In [60]:
def dates_to_idx(timelist):
    reference_time = pd.to_datetime("1870-01-01")
    t = (timelist - reference_time) / pd.Timedelta(365, "D")
    return np.asarray(t)
t = dates_to_idx(df.index)

y = df['NINA12'].values
first_nina = y[0]
y_std = np.std(y)
y_n = (y - first_nina) / y_std

df = df.assign(t=t)
df = df.assign(y_n=y_n)

In [62]:
t = df['t'].values[:, None]
y = df['y_n'].values

In [75]:
with pm.Model() as model:
    # First periodic
    n_per1 = pm.HalfCauchy('n_per1', beta=1.5, initval=0.5)
    l_per1 = pm.Gamma('l_per1', alpha=1, beta=2)
    period_1 = pm.Normal('period_1', mu=1.5, sigma=5, initval=1)
    cov_per1 = n_per1**2 * pm.gp.cov.Periodic(1, period_1, l_per1)
    gp_per1 = pm.gp.Marginal(cov_func=cov_per1)

    # Second periodic
    n_per2 = pm.HalfCauchy('n_per2', beta=1.5, initval=0.5)
    l_per2 = pm.Gamma('l_per2', alpha=1, beta=2)
    period_2 = pm.Normal('period_2', mu=12, sigma=5, initval=1)
    cov_per2 = n_per1**2 * pm.gp.cov.Periodic(1, period_2, l_per2)
    gp_per2 = pm.gp.Marginal(cov_func=cov_per2)

    # gp short
    η_short = pm.HalfCauchy("η_short", beta=1, initval=0.5)
    ℓ_short = pm.Gamma("ℓ_short", alpha=2, beta=10)
    cov_short = η_short**2 * pm.gp.cov.ExpQuad(1, ℓ_short)
    gp_short = pm.gp.Marginal(cov_func=cov_short)

    # noise model
    σ = pm.HalfNormal("σ", sigma=0.25, initval=0.1)
    # The Gaussian process is a sum of these three components
    gp = gp_per1 + gp_per2 + gp_short

    # Since the normal noise model and the GP are conjugates, we use `Marginal` with the `.marginal_likelihood` method
    y_ = gp.marginal_likelihood("y", X=t, y=y, sigma=σ)

    # this line calls an optimizer to find the MAP
    mp = pm.find_MAP(include_transformed=True)

Output()

In [64]:
# display the results, dont show transformed parameter values
sorted([name + ":" + str(mp[name]) for name in mp.keys() if not name.endswith("_")])

['η_short:1.50034738068188',
 'σ:0.34699468056912286',
 'ℓ_short:0.36617555208135505']

In [73]:
dates = pd.date_range(start="1870-01-01", end="08/01/2040", freq="30D")
tnew = dates_to_idx(dates)[:, None]

print("Sampling gp predictions ...")
with model:
    mu_pred, cov_pred = gp.predict(tnew, point=mp)

# draw samples, and rescale
n_samples = 5
samples = pm.draw(pm.MvNormal.dist(mu=mu_pred, cov=cov_pred), draws=n_samples)
samples = samples * y_std + first_nina

Sampling gp predictions ...


In [74]:
# make plot
p = figure(x_axis_type="datetime", width=700, height=300)
p.yaxis.axis_label = "Nino 1.2"
p.xaxis.axis_label = "Date"

# plot mean and 2σ region of total prediction
# scale mean and var
mu_pred_sc = mu_pred * y_std + first_nina
sd_pred_sc = np.sqrt(np.diag(cov_pred) * y_std**2)

upper = mu_pred_sc + 2 * sd_pred_sc
lower = mu_pred_sc - 2 * sd_pred_sc
band_x = np.append(dates, dates[::-1])
band_y = np.append(lower, upper[::-1])

p.line(dates, mu_pred_sc, line_width=2, line_color="firebrick", legend_label="Total fit")
p.patch(band_x, band_y, color="firebrick", alpha=0.6, line_color="white")

# some predictions
idx = np.random.randint(0, samples.shape[0], 10)
p.multi_line(
    [dates] * len(idx),
    [samples[i, :] for i in idx],
    color="firebrick",
    alpha=0.5,
    line_width=0.5,
)

# true value
# p.circle(data_later.index, data_later["CO2"], color="black", legend_label="Observed data")
p.scatter(
    df.index,
    df["Date"],
    marker="circle",
    line_color="black",
    alpha=0.1,
    size=5,
    legend_label="Observed data",
)

p.legend.location = "bottom_right"
show(p)